In [1]:
import pandas as pd
from arcgis.gis import GIS
from arcgis.geocoding import geocode

The version of the binaries in the active Python environment do
  not match the version of the binaries of the ArcGIS Pro application.

Active environment version : 2.8
arcgispro-py3 environment version : 2.9

To create a new environment that is up to date with arcgispro-py3:
 1. Generate a list of additional packages installed in your current environment,
    with the conda command:
        > conda env export > env.yaml
 2. (Optional) If you have additional dependencies installed through pip,
    find those with pip freeze:
        > pip freeze > requirements.txt
 3. Create a new environment by cloning arcgispro-py3, and activate it:
        > conda create --clone arcgispro-py3 --name my-env --pinned
        > activate my-env
 4. Add back missing conda packages from your current environment:
        > conda env update -n my-env -f env.yaml
 5. (Optional) Add back missing packages from pip:
        > pip install -r requirements.txt
 6. (Optional) Make it the default for the ArcGIS Pro 

The version of the binaries in the active Python environment do
  not match the version of the binaries of the ArcGIS Pro application.

Active environment version : 2.8
arcgispro-py3 environment version : 2.9

To create a new environment that is up to date with arcgispro-py3:
 1. Generate a list of additional packages installed in your current environment,
    with the conda command:
        > conda env export > env.yaml
 2. (Optional) If you have additional dependencies installed through pip,
    find those with pip freeze:
        > pip freeze > requirements.txt
 3. Create a new environment by cloning arcgispro-py3, and activate it:
        > conda create --clone arcgispro-py3 --name my-env --pinned
        > activate my-env
 4. Add back missing conda packages from your current environment:
        > conda env update -n my-env -f env.yaml
 5. (Optional) Add back missing packages from pip:
        > pip install -r requirements.txt
 6. (Optional) Make it the default for the ArcGIS Pro 

In [18]:
Halifax_GIS_Map = GIS(username = "JuanCReyes1", password="Kamituru1.")

places_dict= {"Dartmouth":"Dartmouth,Nova Scotia,Canada",
              "Halifax": "Halifax, Nova Scotia, Canada",
              "Bedford": "Bedford, Nova Scotia, Canada"
              }
Halifax_Covid_Exposures = Halifax_GIS_Map.map(places_dict["Dartmouth"],12)
Halifax_Covid_Exposures.basemap = "streets-night-vector" #map style
Halifax_Covid_Exposures

MapView(layout=Layout(height='400px', width='100%'))

In [3]:
import pandas as pd
from arcgis.gis import GIS
from arcgis.geocoding import geocode
#Data Cleaning ###Filter Flights and locate them at the Halifax Stanfield International Airport

#Import Nova Scotia Covid Data from NSHealth.ca
df = pd.read_html("http://www.nshealth.ca/covid-exposures")
df = df[0]

##Check For NaN values
if df.isnull().values.any():
    print("Check for NaN values in our columns")
    print("Place: " + str(df["Place"].isnull().values.any()))
    print("Potential Exposure Window: " + str(df["Potential Exposure Window"].isnull().values.any()))
    print("Address: " + str(df["Address"].isnull().values.any()))
    print("Details: " + str(df["Details"].isnull().values.any()))
    print("Advice: " + str(df["Advice"].isnull().values.any()))
    print("Zone: " + str(df["Zone"].isnull().values.any()))
    print("Last Updated: " + str(df["Last Updated"].isnull().values.any()))

##Zone Column NAN
if df['Zone'].isnull().values.any():
    print("Zone value empty due to flights")
    null_zone_index = df[df['Zone'].isnull().values].index
    df.iloc[null_zone_index,5] = "Flight"
    print("Zone Data Cleaning Complete: "+ str(df["Zone"].isnull().any()))
    
#Identify Covid Flights
print("Attributing Covid Flights to Halifax Stanfield International Airport")
covid_flights = df[(df["Place"].str.contains("flight")) | (df["Place"].str.contains("Flight") ) ]
#Update Covid Flights Address to Halifax Stanfield International Airport:
df["Address"].iloc[covid_flights.index] = "Halifax Stanfield International Airport" +"-"+ df["Address"].iloc[covid_flights.index]

#Create a Geolocation Address which will be fed into ArcGIS
print("Creating A Geolocation Address Column For Each Exposure Site...")
df['Geolocation Address'] = df[['Address', 'Place']].apply(lambda x: '-'.join(x.map(str)), axis=1)
print("Geolocation Address Column: Included")

print("Creating Geolocation Address Coordinates")
##Create GIS Object!
print("Creating ArcGIS Map Object")

df['Geolocation Address Coords'] = df['Geolocation Address'].apply(lambda x: geocode(x,out_fields="address,location,City,Country,URL"))
print("Complete")

##Obtaining Latitude and Longitude for each address:
print("Create Latitude & Longitude Columns")
indices = df['Geolocation Address Coords'].index
longitudeseries = []
latitudeseries = []
print("Creating Latitude and Longitude Columns")
for number in indices:
    longitudeseries.append(df['Geolocation Address Coords'][number][0]["location"]["x"])
    latitudeseries.append(df['Geolocation Address Coords'][number][0]["location"]["y"])

df["longitude"] = longitudeseries
df["latitude"] = latitudeseries

##Export to CSV
print("Exporting to CSV")
df.to_csv('./covidlocations.csv')
print("CSV Complete")

Attributing Covid Flights to Halifax Stanfield International Airport
Creating A Geolocation Address Column For Each Exposure Site...
Geolocation Address Column: Included
Creating Geolocation Address Coordinates
Creating ArcGIS Map Object
Complete
Create Latitude & Longitude Columns
Creating Latitude and Longitude Columns
Exporting to CSV
CSV Complete


In [19]:
import pandas as pd
import pandas as pd
from arcgis.gis import GIS
from arcgis.geocoding import geocode

#1 data import
df1 = pd.read_html("http://www.nshealth.ca/covid-exposures")
df2 = pd.read_html("https://www.nshealth.ca/coronavirus-exposures?title=&field_covid_exposure_zone_value=All&page=1")
df_transit = pd.read_html("https://www.nshealth.ca/coronavirus-exposures-transit")
df_transit = df_transit[0].rename(columns={'Route or flight': "Place"})
df3 = pd.concat([df1[0],df2[0],df_transit], ignore_index = True)


#2 data cleaning
##Check For NaN values
if df3.isnull().values.any():
    print("Check for NaN values in our columns")
    print("Place: " + str(df3["Place"].isnull().values.any()))
    print("Potential Exposure Window: " + str(df3["Potential Exposure Window"].isnull().values.any()))
    print("Address: " + str(df3["Address"].isnull().values.any()))
    print("Details: " + str(df3["Details"].isnull().values.any()))
    print("Zone: " + str(df3["Zone"].isnull().values.any()))
    print("Last Updated: " + str(df3["Last Updated"].isnull().values.any()))
    print("Exposure Area: " + str(df3["Exposure area"].isnull().values.any()))
#print("no null values")

#Exposure Area NAN

if df3["Exposure area"].isnull().values.any():
    print("Empty values in exposure area")
    null_exposures_index = df3[df3["Exposure area"].isnull().values].index
    df3["Exposure area"].iloc[null_exposures_index] = df3["Place"].iloc[null_exposures_index]


##Zone Column NAN
if df3['Zone'].isnull().values.any():
    print("Zone value empty due to flights")
    null_zone_index = df3[df3['Zone'].isnull().values].index
    df3.iloc[null_zone_index,4] = "Flight"
    print("Zone Data Cleaning Complete: "+ str(df3["Zone"].isnull().any()))
#print("no null zone values")


#Identify Covid Flights and Transit Routes
print("Attributing Covid Flights to Halifax Stanfield International Airport")
covid_flights = df3[df3["Place"].str.contains("to Halifax")]
covid_bus_routes = df3[(df3["Place"].str.contains("Transit")) | (df3["Place"].str.contains("Route"))]
#Update Covid Flights Address to Halifax Stanfield International Airport:
df3["Address"].iloc[covid_flights.index] = "Halifax Stanfield International Airport"
df3["Zone"].iloc[covid_flights.index] = "Flight"

#Update Covid Routes Address to the Transit Route
df3["Address"].iloc[covid_bus_routes.index] = "Halifax Transit Bridge Terminal"
#df3["Zone"].iloc[covid_bus_routes.index] = "Halifax Transit Bridge Terminal"

#Create a Geolocation Address which will be fed into ArcGIS
print("Creating A Geolocation Address Column For Each Exposure Site...")
df3['Geolocation Address'] = df3[['Place', 'Address']].apply(lambda x: '-'.join(x.map(str)), axis=1)
print("Geolocation Address Column: Included")



print("Creating Geolocation Address Coordinates")
##Create GIS Object!
print("Creating ArcGIS Map Object")
Halifax_GIS_Map = GIS()

df3['Geolocation Address Coords'] = df3['Geolocation Address'].apply(lambda x: geocode(x,out_fields="address,location,City,Country,URL"))
print("Complete")

##Obtaining Latitude and Longitude for each address:
print("Create Latitude & Longitude Columns")
indices = df3['Geolocation Address Coords'].index
longitudeseries = []
latitudeseries = []

print("Creating Latitude and Longitude Columns")
for number in indices:
    #print(number)
    longitudeseries.append(df3['Geolocation Address Coords'][number][0]["location"]["x"])
    latitudeseries.append(df3['Geolocation Address Coords'][number][0]["location"]["y"])

df3["longitude"] = longitudeseries
df3["latitude"] = latitudeseries

##Export to CSV
print("Exporting to CSV")
df3.to_csv('./covidlocations.csv')
print("CSV Complete")

Check for NaN values in our columns
Place: False
Potential Exposure Window: False
Address: True
Details: False
Zone: True
Last Updated: False
Exposure Area: True
Empty values in exposure area
Zone value empty due to flights
Zone Data Cleaning Complete: False
Attributing Covid Flights to Halifax Stanfield International Airport
Creating A Geolocation Address Column For Each Exposure Site...
Geolocation Address Column: Included
Creating Geolocation Address Coordinates
Creating ArcGIS Map Object
Complete
Create Latitude & Longitude Columns
Creating Latitude and Longitude Columns
Exporting to CSV
CSV Complete


# Map Customization:


### Symbols

In [8]:
geotag_symbol_dict_redbullseye = {"angle":0,"xoffset":0,"yoffset":0,"type":"esriPMS","url":"http://static.arcgis.com/images/Symbols/Basic/esriCrimeMarker_86.png","contentType":"image/png","width":16,"height":16}
geotag_symbol_dict_redfirefly = {"angle":0,"xoffset":0,"yoffset":0,"type":"esriPMS","url":"https://static.arcgis.com/images/Symbols/Firefly/FireflyD20.png","contentType":"image/png","width":24,"height":24}
geotag_symbol_dict_radioactive = {"angle":0,"xoffset":0,"yoffset":0,"type":"esriPMS","url":"http://static.arcgis.com/images/Symbols/PeoplePlaces/Radioactive.png","contentType":"image/png","width":16,"height":16}


### Geotagging

In [9]:
df3

,Place,Potential Exposure Window,Address,Details,Zone,Last Updated,Exposure area,Geolocation Address,Geolocation Address Coords,longitude,latitude
0,Café Marco Polo,"Mon, 11/29/2021 - 10:00 to 11:30","15 Kings Wharf Pl., Dartmouth",details,Central,11/30/2021 - 19:56,Café Marco Polo,"Café Marco Polo-15 Kings Wharf Pl., Dartmouth","[{'address': '15 Kings Wharf Pl, Dartmouth, Ha...",-63.564444,44.664205
1,Access Nova Scotia,"Mon, 11/29/2021 - 10:00 to 13:00","300 Horseshoe Lake Dr., Halifax",details,Central,11/30/2021 - 19:58,Access Nova Scotia,"Access Nova Scotia-300 Horseshoe Lake Dr., Hal...","[{'address': '300 Horseshoe Lake Dr, Halifax, ...",-63.671758,44.638297
2,Pitch & Pivot,"Sun, 11/28/2021 - 09:30 to 11:30","240 Brownlow Ave., Dartmouth",details,Central,11/30/2021 - 19:55,Pitch & Pivot,"Pitch & Pivot-240 Brownlow Ave., Dartmouth","[{'address': '240 Brownlow Ave, Dartmouth, Hal...",-63.578394,44.694993
3,The Old Triangle Irish Alehouse,"Sat, 11/27/2021 - 19:00 to 23:00","5136 Prince St, Halifax",details,Central,11/30/2021 - 19:45,The Old Triangle Irish Alehouse,The Old Triangle Irish Alehouse-5136 Prince St...,"[{'address': 'Old Triangle Irish Alehouse', 'l...",-63.572300,44.647670
4,Moxies Bar & Grill,"Sat, 11/27/2021 - 16:30 to 18:30","9 Countryview Dr., Dartmouth",details,Central,11/30/2021 - 19:50,Moxies Bar & Grill,"Moxies Bar & Grill-9 Countryview Dr., Dartmouth","[{'address': 'Moxie's', 'location': {'x': -63....",-63.564910,44.700980
...,...,...,...,...,...,...,...,...,...,...,...
90,Halifax Transit Route #001 (Spring Garden),"Mon, 11/15/2021 - 09:45 to 10:15",Halifax Transit Bridge Terminal,details,Central,11/20/2021 - 20:09,Bridge Terminal to Mumford Terminal,Halifax Transit Route #001 (Spring Garden)-Hal...,"[{'address': 'Spring Gardens, Norwood Green, H...",-1.796058,53.738246
91,Air Canada 674 Calgary to Halifax,"Mon, 11/15/2021 - 09:30 to 17:45",Halifax Stanfield International Airport,details,Flight,11/17/2021 - 18:16,"Passengers in rows 26-32, seats A, B, C and D.",Air Canada 674 Calgary to Halifax-Halifax Stan...,[{'address': 'Halifax Stanfield International ...,-63.515310,44.886260
92,Halifax Transit Route 320 Regional Express Air...,"Fri, 11/12/2021 - 22:15 to 23:00",Halifax Transit Bridge Terminal,details,Central,11/18/2021 - 19:07,from Halifax Stanfield International Airport t...,Halifax Transit Route 320 Regional Express Air...,"[{'address': 'Fall River, Halifax, Nova Scotia...",-63.611430,44.816340
93,Air Canada 624 from Toronto to Halifax,"Fri, 11/12/2021 - 20:45 to 23:45",Halifax Stanfield International Airport,Details,Flight,11/15/2021 - 10:07,"Passengers in rows 29 to 35, seats A, C and D",Air Canada 624 from Toronto to Halifax-Halifax...,[{'address': 'Halifax Stanfield International ...,-63.515310,44.886260


In [20]:
#plot covid locations
for number in range(0,len(df3['Geolocation Address Coords'])):
#for number in range(0,15):
    location = df3['Geolocation Address Coords'][number][0]["location"]
    popup = {"title": pd.Series(df3["Geolocation Address"])[number], "content": pd.Series(df3["Exposure area"])[number]}
    symbol = geotag_symbol_dict_radioactive
    attributes = {"test": "what"}
    Halifax_Covid_Exposures.draw(shape = location,popup=popup, symbol = symbol ,attributes = "test")
    print("Plotting location number " + str(number) + "/ " +str(len(df3["Geolocation Address Coords"])-1)+": " + pd.Series(df3["Geolocation Address"])[number] )

    
#also plot all healthcare facilities in canada    
covid_canada = Halifax_GIS_Map.content.search("title: Open Database of Healthcare Facilities - Canada")
covid_canada[0]
covid_layer = Halifax_GIS_Map.content.get(covid_canada[0].id)
Halifax_Covid_Exposures.add_layer(covid_layer,options={'opacity':1.2})

Plotting location number 0/ 94: Café Marco Polo-15 Kings Wharf Pl., Dartmouth
Plotting location number 1/ 94: Access Nova Scotia-300 Horseshoe Lake Dr., Halifax
Plotting location number 2/ 94: Pitch & Pivot-240 Brownlow Ave., Dartmouth
Plotting location number 3/ 94: The Old Triangle Irish Alehouse-5136 Prince St, Halifax
Plotting location number 4/ 94: Moxies Bar & Grill-9 Countryview Dr., Dartmouth
Plotting location number 5/ 94: Ela! Greek Taverna-150 Hector Gate, Dartmouth
Plotting location number 6/ 94: Sou’Wester Gift and Restaurant-178 Peggys Point Rd., Peggys Cove
Plotting location number 7/ 94: Hermitage-1460 Lower Water St., Halifax
Plotting location number 8/ 94: Pür & Simple-189 Hector Gate, Dartmouth
Plotting location number 9/ 94: Pitch & Pivot-240 Brownlow Ave., Dartmouth
Plotting location number 10/ 94: Hermitage-1460 Lower Water St., Halifax
Plotting location number 11/ 94: GoodLife Fitness-3711 Joseph Howe Dr., Halifax
Plotting location number 12/ 94: Hermitage-1460 L

Plotting location number 92/ 94: Halifax Transit Route 320 Regional Express Airport to Fall River-Halifax Transit Bridge Terminal
Plotting location number 93/ 94: Air Canada 624 from Toronto to Halifax-Halifax Stanfield International Airport
Plotting location number 94/ 94: Air Canada 8068 from Ottawa to Halifax-Halifax Stanfield International Airport


In [ ]:
covid_canada = Halifax_GIS_Map.content.search("title: Open Database of Healthcare Facilities - Canada")
covid_canada[0]
#covid_layer = Halifax_GIS_Map.content.get(covid_canada[0].id)
Halifax_Covid_Exposures.add_layer(covid_layer,options={'opacity':1.2})

In [14]:
covid_canada[0]

<Item title:"Open Database of Healthcare Facilities" type:Feature Layer Collection owner:pheersink_exchange>

# TESTING GROUNDS

In [14]:
Halifax_GIS_Map = GIS(username="JuanCReyes1")

places_dict= {"Dartmouth":"Dartmouth,Nova Scotia,Canada",
              "Halifax": "Halifax, Nova Scotia, Canada",
              "Bedford": "Bedford, Nova Scotia, Canada"
              }
Halifax_Covid_Exposures = Halifax_GIS_Map.map(places_dict["Dartmouth"],12)
Halifax_Covid_Exposures.basemap = "streets-night-vector" #map style
Halifax_Covid_Exposures

Enter password: ········


MapView(layout=Layout(height='400px', width='100%'))

In [15]:
for number in range(0,len(df['Geolocation Address Coords'])):
#for number in range(0,15):
    location = df['Geolocation Address Coords'][number][0]["location"]
    popup = {"title": pd.Series(df["Geolocation Address"])[number], "content": pd.Series(df["Advice"])[number]}
    symbol = geotag_symbol_dict_redbullseye
    attributes = {"test": "what"}
    Halifax_Covid_Exposures.draw(shape = location,popup=popup, symbol = symbol ,attributes = "test")
    print("Plotting location number " + str(number) + "/ " +str(len(df["Geolocation Address Coords"])-1)+": " + pd.Series(df["Geolocation Address"])[number] )

covid_canada = Halifax_GIS_Map.content.search("title: Open Database of Healthcare Facilities - Canada")
covid_canada[0]
covid_layer = Halifax_GIS_Map.content.get(covid_canada[0].id)
Halifax_Covid_Exposures.add_layer(covid_layer,options={'opacity':1.2})


Plotting location number 0/ 33: Halifax Stanfield International Airport-Rows 21-27 in seats A, B, C and D-Air Canada flight 622 from Toronto to Halifax
Plotting location number 1/ 33: Halifax Stanfield International Airport-Rows 26-32 in seats C, D, E and F-Air Canada flight 614 from Toronto to Halifax
Plotting location number 2/ 33: 1120 Queen St., Halifax-Sobeys
Plotting location number 3/ 33: 1075 Barrington St., Halifax-Superstore
Plotting location number 4/ 33: Halifax Stanfield International Airport-Rows 16-22 in seats A, B, C and D-Air Canada flight 604 from Toronto to Halifax
Plotting location number 5/ 33: Halifax Stanfield International Airport-Rows 15-21 in seats C, D, E and F and rows 1-4 in seats A, C, and D-Air Canada flight 624 from Toronto to Halifax
Plotting location number 6/ 33: Halifax Stanfield International Airport-Rows 1-4, seats A, C and D-WestJet flight 248 from Toronto to Halifax
Plotting location number 7/ 33: Halifax Stanfield International Airport-Rows 1-4,

In [15]:
from arcgis.gis import GIS
myGIS = GIS(username = "Juan_Reyes_LearnArcGIS", password="Kamituru1.")
Halifax_Covid_Exposures = myGIS.map(places_dict["Dartmouth"],12)
Halifax_Covid_Exposures.basemap = "streets-night-vector" #map style

covid_canada = myGIS.content.search("title: Open Database of Healthcare Facilities - Canada")
covid_canada

[]

# OTHER

In [16]:
from arcgis.gis import GIS
myGIS = GIS(username = "Juan_Reyes_LearnArcGIS", password="Kamituru1.")
Halifax_Covid_Exposures =myGIS.map(places_dict["Dartmouth"],12)
Halifax_Covid_Exposures.basemap = "streets-night-vector" #map style

places_dict= {"Dartmouth":"Dartmouth,Nova Scotia,Canada",
              "Halifax": "Halifax, Nova Scotia, Canada",
              "Bedford": "Bedford, Nova Scotia, Canada"
              }

#myMap = myGIS.map(places_dict["Dartmouth"],10)
#myMap.basemap = "dark-gray-vector" #map style

geotag_symbol_dict_radioactive= {"angle":0,"xoffset":0,"yoffset":0,"type":"esriPMS","url":"http://static.arcgis.com/images/Symbols/PeoplePlaces/Radioactive.png","contentType":"image/png","width":16,"height":16}


covid_canada = myGIS.content.search("title: Open Database of Healthcare Facilities - Canada")
covid_canada[0]
covid_layer = myGIS.content.get(covid_canada[0].id)
myMap.add_layer(covid_layer)
myMap.display_message("hello world")
myMap.draw([44.6476,-63.57],symbol=geotag_symbol_dict_radioactive)


IndexError: list index out of range

In [ ]:
#from arcgis.features import from_xy
from arcgis.features import GeoAccessor
spatialdf = GeoAccessor.from_xy(df = df2, x_column="longitude", y_column="latitude")

In [ ]:
df2=df.copy()

In [ ]:
spatialdf["SHAPE"]

In [ ]:
from arcgis.features import GeoAccessor

In [ ]:
batch_geocode_covid = batch_geocode(addresses=df["Geolocation Address"])

In [ ]:
from arcgis.features import to_feature_collection

In [ ]:
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor